In [ ]:
# CSV 파일의 내용을 정리하고 형식을 맞추는 코드
import csv
import re

def clean_text(text):
    # HTML 태그 제거
    clean = re.sub(r'<[^>]+>', '', text)
    # 메뉴 항목 제거 (HOME, LOGIN 등)
    clean = re.sub(r'\b(HOME|LOGIN)\b', '', clean)
    # 중복된 메뉴 항목 제거
    clean = re.sub(r'(학 부.*?자료실\s*){2,}', r'\1', clean)
    # 연속된 공백 제거
    clean = re.sub(r'\s+', ' ', clean)
    # 연속된 빈 줄을 하나로 줄이기
    clean = re.sub(r'\n\s*\n', '\n', clean)
    # 앞뒤 공백 제거
    clean = clean.strip()
    return clean

def clean_and_format_csv():
    input_file = 'sejong_content.csv'
    output_file = 'sejong_content_clean.csv'
    
    with open(input_file, 'r', encoding='utf-8') as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        rows = list(reader)
    
    with open(output_file, 'w', encoding='utf-8-sig', newline='') as fout:
        writer = csv.writer(fout, quoting=csv.QUOTE_ALL)
        writer.writerow(headers)
        
        for row in rows:
            # 각 필드 정리
            cleaned_row = [clean_text(field) for field in row]
            writer.writerow(cleaned_row)
    
    print(f"CSV 파일이 성공적으로 정리되었습니다: {output_file}")

if __name__ == '__main__':
    clean_and_format_csv()


CSV 파일이 성공적으로 정리되었습니다: sejong_content_clean.csv


In [ ]:
import pandas as pd

# 엑셀에서 불러오기 (1행짜리 데이터라고 가정)
df = pd.read_csv(r"C:\Users\good1\Desktop\202501\mcptest\withclaud\sejong_content_cleans.csv", header=None)

# 1행을 리스트로 가져오기
data = df.iloc[0].tolist()

# 7개씩 끊어서 행으로 재배치
reshaped = [data[i:i+6] for i in range(0, len(data), 6)]

# 새 DataFrame 만들기
new_df = pd.DataFrame(reshaped)

# 저장
new_df.to_excel('reshaped.xlsx', index=False, header=False)

In [ ]:
    # 분류 작업을 위한 Ollama 모델을 사용하여 엑셀 파일의 내용을 분류하는 코드 세종대학교 연구용 0번 데이터
import pandas as pd
import ollama
from tqdm import tqdm

# Load reshaped.xlsx
df = pd.read_excel('reshaped.xlsx')

# Ensure the columns exist
assert '타이틀' in df.columns and '본문내용' in df.columns, "Excel file must contain '타이틀' and '본문내용' columns"

# 분류 라벨
categories = ['professor', 'members', 'research', 'publications', 'lectures', 'projects', 'events', 'recruit', 'others']

# 결과 저장 컬럼
df['분류'] = ''
    
# Prompt template
def make_prompt(title, content):
    return f"""
아래는 홈페이지 섹션에 대한 정보입니다.

타이틀: {title}
내용: {content}

이 정보가 어떤 범주에 속하는지 다음 중 하나로 분류해 주세요:
{', '.join(categories)}.

오직 분류 이름만 출력하세요.
"""

# Ollama 분류 요청
def classify_row(title, content):
    prompt = make_prompt(title, content)
    try:
        response = ollama.chat(model='gemma3:12b', messages=[
            {'role': 'user', 'content': prompt}
        ])
        return response['message']['content'].strip().lower()
    except Exception as e:
        print("Error:", e)
        return 'others'

# 진행
for idx, row in tqdm(df.iterrows(), total=len(df)):
    title, content = row['타이틀'], row['본문내용']
    label = classify_row(title, content)
    df.at[idx, '분류'] = label if label in categories else 'others'

# 결과 저장
df.to_excel('classified_reshaped.xlsx', index=False)
print("✅ 분류 완료. 'classified_reshaped.xlsx'로 저장됨.")


100%|██████████| 137/137 [01:05<00:00,  2.09it/s]

✅ 분류 완료. 'classified_reshaped.xlsx'로 저장됨.


In [ ]:
# 분류 작업을 위한 Ollama 모델을 사용하여 엑셀 파일의 내용을 분류하는 코드  1~6 데이터
import pandas as pd
import ollama
from tqdm import tqdm

# Load reshaped.xlsx
df = pd.read_excel('university6.xlsx') #1~6

# Ensure the columns exist
assert '메뉴경로' in df.columns and '내용' in df.columns, "Excel file must contain '메뉴경로' and '내용' columns"

# 분류 라벨
categories = ['professor', 'members', 'research', 'publications', 'lectures', 'projects', 'events', 'recruit', 'others']

# 결과 저장 컬럼
df['분류'] = ''

# Prompt template
def make_prompt(title, content):
    return f"""
아래는 홈페이지 섹션에 대한 정보입니다.

타이틀: {title}
내용: {content}

이 정보가 어떤 범주에 속하는지 다음 중 하나로 분류해 주세요:
{', '.join(categories)}.

오직 분류 이름만 출력하세요.
"""

# Ollama 분류 요청
def classify_row(title, content):
    prompt = make_prompt(title, content)
    try:
        response = ollama.chat(model='gemma3:12b', messages=[
            {'role': 'user', 'content': prompt}
        ])
        return response['message']['content'].strip().lower()
    except Exception as e:
        print("Error:", e)
        return 'others'

# 진행  
for idx, row in tqdm(df.iterrows(), total=len(df)):
    title, content = row['메뉴경로'], row['내용']
    label = classify_row(title, content)
    df.at[idx, '분류'] = label if label in categories else 'others'

# 결과 저장
df.to_excel('university6_ai.xlsx', index=False)
print("✅ 분류 완료. 'university6_ai.xlsx'로 저장됨.")


100%|██████████| 98/98 [00:26<00:00,  3.70it/s]

✅ 분류 완료. 'university6_ai.xlsx'로 저장됨.


In [ ]:
import pandas as pd
import ollama
from tqdm import tqdm

# Load Excel file
df = pd.read_excel('university_total.xlsx')
assert '타이틀' in df.columns and '본문내용' in df.columns and '분류' in df.columns, "필수 컬럼이 없습니다."

# 결과 저장 컬럼
df['페이지내용정리'] = ''

# 카테고리 정의
categories = ['professor', 'members', 'research', 'publications', 'lectures', 'projects', 'events', 'recruit']

# 각 카테고리에 맞는 프롬프트 작성
def make_prompt(category, content):
    instructions = {
        'professor': "이 교수님의 대표 논문과 주요 연구 내용을 한 문장으로 요약해 주세요.",
        'members': (
            "연구실 구성원(학생) 정보를 아래 항목을 기준으로 정리해 주세요.\n"
            "학생 이름 / 근속 년수(있으면) / 연구분야(있으면) / 연구주제(있으면) / 기타사항(논문, 수상 등)\n"
            "가능하면 리스트 형태로 주세요."
        ),
        'research': "이 연구실이 수행 중인 연구 주제를 한 문장으로 요약해 주세요.",
        'publications': "연구실의 주요 논문 또는 출판물을 리스트 형식으로 정리해 주세요.",
        'lectures': "교수님이 가르친 과목들을 리스트 형식으로 정리해 주세요.",
        'projects': (
            "연구실에서 수행한 프로젝트를 정리해 주세요. 각 항목마다 아래 형식으로 작성해 주세요:\n"
            "프로젝트명: 설명 / 어떤 일을 수행했는지\n"
            "각 프로젝트는 줄바꿈으로 구분해 주세요."
        ),
        'recruit': "연구실에서 모집하는 학생의 조건이나 방향을 한 문장으로 정리해 주세요.",
        'events': "행사의 종류와 주요 내용(언제, 무슨 행사)을 간단히 정리해 주세요."
    }
    instruction = instructions.get(category, "해당 내용을 간단히 정리해 주세요.")
    
    return f"""[홈페이지 내용 분석]

카테고리: {category}
내용:
{content}

요청사항:
{instruction}
"""

# Ollama inference
def classify_row(category, content):
    prompt = make_prompt(category, content)
    try:
        response = ollama.chat(model='gemma3:12b', messages=[
            {'role': 'user', 'content': prompt}
        ])
        return response['message']['content'].strip()
    except Exception as e:
        print(f"❌ Error at category [{category}]:", e)
        return '분석 실패'

# 실행
for idx, row in tqdm(df.iterrows(), total=len(df)):
    category, content = row['분류'], row['본문내용']
    result = classify_row(category, content)
    df.at[idx, '페이지내용정리'] = result

# 저장
df.to_excel('university_total_refined.xlsx', index=False)
print("✅ 정리 완료. 'university_total_refined.xlsx'로 저장됨.")


100%|██████████| 1424/1424 [3:16:24<00:00,  8.28s/it]  


✅ 정리 완료. 'university_total_refined.xlsx'로 저장됨.


In [1]:
import pandas as pd
import subprocess
import json
from tqdm import tqdm

# 1. 처리할 분류 목록 및 프롬프트 템플릿 정의
PROMPT_TEMPLATES = {
    "members": """다음은 "members" 섹션에 있는 텍스트야. 이 안에서 학생 이름, 근속 년수, 연구분야, 연구주제, 기타사항을 정리해서 아래와 같은 표로 변환해줘:

형식:
| 학생 이름 | 근속 년수 | 연구분야 | 연구주제 | 기타사항 |
|-----------|------------|-----------|-----------|------------|

출력은 오직 위 표 형식으로만 해줘. 다른 설명은 하지 마. 텍스트는 다음과 같아:

---
{content}
""",
    "publications": """다음은 "publications" 섹션의 텍스트야. 아래 형식으로 논문 리스트를 표로 변환해줘:

형식:
| 논문 이름 | 저널 | 저자 | 연도 |
|-----------|--------|--------|------|

출력은 위 표로만 해줘. 불필요한 설명 없이.

텍스트:
---
{content}
""",
    "lectures": """다음은 "lectures" 섹션 텍스트야. 여기에 언급된 강의 과목명을 리스트 형태로 변환해줘:

형식:
["과목1", "과목2", "과목3", ...]

텍스트:
---
{content}
""",
    "projects": """다음은 "projects" 섹션이야. 아래와 같은 테이블 형식으로 변환해줘:

| 프로젝트명 | 프로젝트 설명 |
|-------------|------------------|

출력은 테이블 형태로만 해줘. 텍스트:
---
{content}
""",
}

def query_ollama(prompt, model='gemma3:12b'):
    """Ollama CLI 호출하여 결과 반환"""
    try:
        result = subprocess.run(
            ['ollama', 'run', model],
            input=prompt,
            text=True,
            capture_output=True,
            timeout=60
        )
        return result.stdout.strip()
    except subprocess.TimeoutExpired:
        return "타임아웃 오류"
    except Exception as e:
        return f"에러 발생: {str(e)}"

# 2. 엑셀 파일 불러오기
input_path = 'university_total_refined.xlsx'
output_path = '연구실_정보_추출결과2.xlsx'
df = pd.read_excel(input_path)

# 3. 새로운 열 생성
results = []

# 4. tqdm으로 진행상황 출력
for _, row in tqdm(df.iterrows(), total=len(df)):
    category = str(row['분류']).strip().lower()
    content = str(row['페이지내용정리']).strip()

    if category in PROMPT_TEMPLATES:
        prompt = PROMPT_TEMPLATES[category].format(content=content)
        result = query_ollama(prompt)
        results.append(result)
    else:
        results.append("N/A")

# 5. 결과 저장
df['2차정제'] = results
df.to_excel(output_path, index=False)
print(f"저장 완료: {output_path}")


100%|██████████| 1424/1424 [1:25:26<00:00,  3.60s/it] 


저장 완료: 연구실_정보_추출결과2.xlsx


In [19]:
import pandas as pd
import subprocess
from tqdm import tqdm

# 1. 교수 정보 추출용 프롬프트 템플릿 정의
PROFESSOR_PROMPT = """아래 텍스트에서 교수 정보를 표 형식으로 추출해줘:
[교수이름,이메일주소,소속대학교,교수전화번호,연구실위치,기타사항]

형식:
| 교수이름 | 이메일주소 | 소속대학교 | 교수전화번호 | 연구실위치 | 기타사항 |
|-----------|--------|--------|------|

출력은 위 표로만 해줘. 불필요한 설명 없이. 교수이름은 영어로 되어있으면 영어로 출력해줘. 이메일주소는 이메일 형식으로, 전화번호는 숫자만 포함된 형식으로 출력해줘.

텍스트:
---
{content}
"""

def query_ollama(prompt, model='gemma3:12b'):
    """Ollama CLI 호출하여 결과 반환"""
    try:
        result = subprocess.run(
            ['ollama', 'run', model],
            input=prompt,
            text=True,
            capture_output=True,
            timeout=1000
        )
        return result.stdout.strip()
    except subprocess.TimeoutExpired:
        return "타임아웃 오류"
    except Exception as e:
        return f"에러 발생: {str(e)}"

# 2. 엑셀 파일 불러오기
input_path = '연구실_정보_추출결과2.xlsx'
df = pd.read_excel(input_path)

# 3. 분류가 professor인 행만 필터링
df_professor = df[df['분류'].str.strip().str.lower() == 'professor']

# 4. 본문내용에서 정보 추출
results = []
for _, row in tqdm(df_professor.iterrows(), total=len(df_professor)):
    content = str(row['본문내용']).strip()
    prompt = PROFESSOR_PROMPT.format(content=content)
    result = query_ollama(prompt)
    results.append(result)

# 5. 결과 저장 (원본 df에 3차정제 열로 추가)
df_professor['3차정제'] = results
df_professor.to_excel('연구실_정보_추출결과3.xlsx', index=False)
print("저장 완료: 연구실_정보_추출결과3.xlsx")

100%|██████████| 68/68 [04:57<00:00,  4.37s/it]

저장 완료: 연구실_정보_추출결과3.xlsx



C:\Users\good1\AppData\Local\Temp\ipykernel_5868\2146414202.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_professor['3차정제'] = results


In [35]:
import pandas as pd
import numpy as np
import ast
import re
from tqdm import tqdm

# 1. 엑셀 불러오기
df = pd.read_excel("연구실_정보_추출결과2.xlsx")

# 2. '2차정제'가 N/A나 비었으면 '페이지내용정리'로 대체
df['최종정제'] = df['2차정제'].where((df['2차정제'].notna()) & (df['2차정제'] != 'N/A'), df['페이지내용정리'])

# 3. 테이블/리스트 구분 함수
def parse_row_with_missing(row_str, expected_cols):
    # 앞뒤의 '|'를 제거
    row_str = row_str.strip('|')
    # '|'로 split 후 각 항목의 앞뒤 공백 제거
    parts = [part.strip() for part in row_str.split('|')]
    # 빈 문자열을 '미상'으로 대체
    parts = [p if p != '' else '미상' for p in parts]
    # 컬럼 수 맞추기
    if len(parts) < expected_cols:
        parts += ['미상'] * (expected_cols - len(parts))
    elif len(parts) > expected_cols:
        parts = parts[:expected_cols]
    return parts
def extract_table(text):
    if not isinstance(text, str):
        return None, "invalid"
    if '|' in text:
        lines = [line.strip() for line in text.strip().split('\n') if line.strip()]
        if len(lines) < 2:
            return None, "malformed table"
        headers = [col.strip() for col in lines[0].split('|') if col.strip()]
        rows = []
        for line in lines[2:]:
            row = parse_row_with_missing(line, len(headers))
            rows.append(row)
        if not rows:
            return None, "empty table"
        return pd.DataFrame(rows, columns=headers), "table"

    elif text.startswith('[') and text.endswith(']') and ',' in text:
        try:
            raw = text.strip()[1:-1]
            parts = [item.strip() for item in raw.split(',')]
            return parts, "list"
        except Exception as e:
            print(f"[⚠️ 리스트 파싱 오류]: {e}")
            return None, "bad list"

    return None, "other"

# 4. 테이블 리스트 초기화
table1, table2, table3, table4, table5, table6, table7, table8 = [], [], [], [], [], [], [], []

# 5. 행 처리
for idx, row in tqdm(df.iterrows(), total=len(df)):
    main_url = row.get('메인주소', '')
    category = str(row.get('분류', '')).strip().lower()
    title = row.get('타이틀', '')
    url = row.get('주소', '')
    content = row.get('최종정제', '')
    second = row.get('2차정제', '')

    table1.append([main_url, category, title, url])
    
    parsed, kind = extract_table(content)

    if kind == "table" and isinstance(parsed, pd.DataFrame):
        cols = parsed.columns.tolist()
        if cols == ['학생 이름', '근속 년수', '연구분야', '연구주제', '기타사항']:
            for _, r in parsed.iterrows():
                table2.append([main_url, category] + r.tolist())
        elif cols == ['논문 이름', '저널', '저자', '연도']:
            for _, r in parsed.iterrows():
                table3.append([main_url, category] + r.tolist())
        elif cols == ['교수이름', '이메일주소', '소속대학교', '교수전화번호', '연구실위치', '기타사항']:
            for _, r in parsed.iterrows():
                table8.append([main_url, category] + r.tolist())
        elif cols == ['교수님이 가르치시는 과목들']:
            for _, r in parsed.iterrows():
                table4.append([main_url, category, r.tolist()])
        elif cols == ['프로젝트명', '프로젝트 설명']:
            for _, r in parsed.iterrows():
                table5.append([main_url, category] + r.tolist())
        else:
            table7.append([main_url, category, content])

    elif kind == "list" and isinstance(parsed, list):
        if 'lecture' in category:
            # 길이 보정: 최대 20개
            values = parsed[:20]
            values += ['미상'] * (20 - len(values))
            table4.append([main_url, category] + values)
        elif 'professor' in category:
            values = parsed[:6]
            values += [''] * (6 - len(values))
            table8.append([main_url, category] + values)
        else:
            table7.append([main_url, category, content])

    elif kind == "other":
        if 'recruit' in category:
            table6.append([main_url, category, content])
        elif 'research' in category:
            table7.append([main_url, category, content])

# 6. DataFrame 생성
columns1 = ['메인주소', '분류', '타이틀', '주소']
columns2 = ['메인주소', '분류', '연구원 이름', '근속 연수', '연구분야', '연구주제', '기타사항']
columns3 = ['메인주소', '분류', '논문 이름', '저널', '저자', '연도']
columns4 = ['메인주소', '분류'] + [f'과목{i}' for i in range(1, 21)]
columns5 = ['메인주소', '분류', '프로젝트 이름', '프로젝트 설명']
columns6 = ['메인주소', '분류', 'recruit']
columns7 = ['메인주소', '분류', '연구주제']
columns8 = ['메인주소', '분류', '교수이름', '이메일주소', '소속대학교', '교수전화번호', '연구실위치', '기타사항']

df1 = pd.DataFrame(table1, columns=columns1)
df2 = pd.DataFrame(table2, columns=columns2)
df3 = pd.DataFrame(table3, columns=columns3)
df4 = pd.DataFrame(table4, columns=columns4)
df5 = pd.DataFrame(table5, columns=columns5)
df6 = pd.DataFrame(table6, columns=columns6)
df7 = pd.DataFrame(table7, columns=columns7)
df8 = pd.DataFrame(table8, columns=columns8)

# 7. 엑셀 저장
with pd.ExcelWriter("연구실_8개_테이블_정리.xlsx") as writer:
    df1.to_excel(writer, sheet_name="테이블1_기본정보", index=False)
    df2.to_excel(writer, sheet_name="테이블2_연구원", index=False)
    df3.to_excel(writer, sheet_name="테이블3_논문", index=False)
    df4.to_excel(writer, sheet_name="테이블4_교수_과목", index=False)
    df5.to_excel(writer, sheet_name="테이블5_프로젝트", index=False)
    df6.to_excel(writer, sheet_name="테이블6_리크루트", index=False)
    df7.to_excel(writer, sheet_name="테이블7_연구주제", index=False)
    df8.to_excel(writer, sheet_name="테이블8_교수님", index=False)

print("✅ 모든 테이블 정리 및 저장 완료!") 

100%|██████████| 1415/1415 [00:00<00:00, 12938.58it/s]


✅ 모든 테이블 정리 및 저장 완료!
